In [1]:
import numpy as np
import pandas as pd
from random import choice

In [2]:
def generate_junctions(n=10**6):
    a = [()] * n
    for i in range(n):
        seq = 'chr' + str(np.random.randint(1, 23))
        start = str(np.random.randint(1, 10**8+1))
        end = str(np.random.randint(start, 10**8+1))
        strand = choice(("+", "-"))
        offset = np.random.randint(1, 101)
        count = np.random.randint(1, 1001)
        a[i] = ("_".join((seq, start, end, strand)),
               offset, count)
    return a    

In [3]:
junctions = generate_junctions()

In [4]:
from collections import namedtuple
nt_junction = namedtuple("Pandas", ["junction_id", "offset", "count"])

In [5]:
%%timeit
b = []
for junction in junctions:
    b.append(nt_junction(*junction))

246 ms ± 236 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
b = []
for junction in junctions:
    b.append(tuple(iter(junction)))

237 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
b = []
for junction in junctions:
    b.append(list(iter(junction)))

238 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
b = []
for junction in junctions:
    b.append({
        "junction_id": junction[0],
        "offset": junction[1],
        "count": junction[2]
    })

226 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
b = []
for junction in junctions:
    b.append({
        "junction_id": junction[0],
        "offset": junction[1],
        "count": junction[2]
    })

In [10]:
%load_ext memory_profiler

In [11]:
%memit b

peak memory: 542.87 MiB, increment: 0.32 MiB


In [12]:
b = []
for junction in junctions:
    b.append(nt_junction(*junction))

In [13]:
%memit b

peak memory: 418.27 MiB, increment: 0.02 MiB


In [19]:
%memit pd.DataFrame(b)

peak memory: 673.82 MiB, increment: 77.13 MiB


Some recipe from github:

In [19]:
class Record(object):
    
    __slots__ = ()
    
    def __init__(self, *args):
        assert len(self.__slots__) == len(args)
        for field, value in zip(self.__slots__, args):
            setattr(self, field, value)
            
    def __getitem__(self, index):
        return getattr(self, self.__slots__[index])
    
    def __len__(self):
        return len(self.__slots__)
    
    def __eq__(self, that):
        if not isinstance(that, type(self)):
            return NotImplemented
        return all(item == iota for item, iota in zip(self, that))
    
    def __repr__(self):
        args = ", ".join(repr(item) for item in self)
        return f"{type(self).__name__, }({args})"
    
    def __getstate__(self):
        return tuple(self)
    
    def __setstate__(self, state):
        self.__init__(*state)

In [20]:
class re_junction(Record):
    __slots__ = "junction_id", "offset", "count"

In [24]:
%%timeit
b = []
for junction in junctions:
    b.append(re_junction(*junction))

572 ms ± 6.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
b = []
for junction in junctions:
    b.append(re_junction(*junction))

In [26]:
%memit b

peak memory: 409.38 MiB, increment: 0.10 MiB


Сonclusion: use `namedtuple`